<a href="https://colab.research.google.com/github/NomadCoder101/-Django-Practicet-with-CRUD/blob/main/FULL_LIVEWIRE_SENTI_DECSN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installs

In [1]:
# !pip install vaderSentiment
# !pip install textblob
# !pip install nltk
# !pip install pytz
# !pip install urllib3
# !pip install requests
# !pip install beautifulsoup4
# !pip install pandas



In [2]:
# !pip install pandas requests beautifulsoup4 textblob nltk

#Imports

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from datetime import datetime
import pytz
import urllib.parse

In [4]:
import pandas as pd
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

#FULL LIVEWIRE CODE WITH SENTIMENT AND DECISION v1.1

In [5]:
# import pandas as pd
# import requests
# from bs4 import BeautifulSoup
# from textblob import TextBlob
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# import nltk
# from datetime import datetime
# import pytz
# import urllib.parse

# Setup NLTK/VADER
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

# Paths
LEXICON_INPUT = "/path/to/your/sentiment_data.csv"  # Replace with your original lexicon file
LEXICON_PATH = "/content/drive/MyDrive/Livewire/stock_lexicon.csv"
LIVEWIRE_OUTPUT = "/content/drive/MyDrive/Livewire/all_x.csv"

# Step 1: Preprocess Lexicon
def preprocess_lexicon(input_file):
    print("=== Preprocessing Lexicon ===")
    df = pd.read_csv(input_file)
    trading_terms = [
        "breakout", "crash", "bullish", "bearish", "support", "resistance", "squeeze",
        "rally", "dip", "surge", "volatility", "trend", "buy", "sell", "market", "stock", "price"
    ]
    df_filtered = df[
        df['Item'].str.lower().isin(trading_terms) |
        df['Item'].str.contains("stock|trade|market|price|buy|sell", case=False, na=False)
    ].copy()

    df_filtered.loc[:, 'Aff_Score'] = df_filtered['Aff_Score'].clip(lower=-5, upper=5) / 5
    df_filtered.loc[:, 'Neg_Score'] = df_filtered['Neg_Score'].clip(lower=-5, upper=5) / 5
    df_filtered.loc[:, 'Sentiment_Score'] = (df_filtered['Aff_Score'] + df_filtered['Neg_Score']) / 2

    df_filtered[['Item', 'POS', 'Sentiment_Score']].to_csv(LEXICON_PATH, index=False)
    print(f"Saved {len(df_filtered)} terms to {LEXICON_PATH}")
    return df_filtered

# Scraping Functions
def scrape_google_news():
    print("\n=== Scraping Google News ===")
    google_data = []
    url = "https://news.google.com/rss/search?q=SPY+OR+TSLA&hl=en-US&gl=US&ceid=US:en"
    try:
        response = requests.get(url, timeout=30, headers={'User-Agent': 'Mozilla/5.0'})
        print(f"Google News response status: {response.status_code}")
        soup = BeautifulSoup(response.text, 'xml')
        items = soup.find_all('item')[:10]
        print(f"Found {len(items)} Google News RSS items")
        for item in items:
            title = item.title.text if item.title else "No title"
            pub_date = item.pubDate.text if item.pubDate else datetime.now(pytz.utc)
            if isinstance(pub_date, str):
                try:
                    pub_date = datetime.strptime(pub_date, "%a, %d %b %Y %H:%M:%S %z")
                except ValueError:
                    pub_date = datetime.strptime(pub_date, "%a, %d %b %Y %H:%M:%S GMT")
                    pub_date = pub_date.replace(tzinfo=pytz.UTC)
            pdt = pytz.timezone('America/Los_Angeles')
            timestamp = pub_date.astimezone(pdt).strftime('%Y-%m-%d %H:%M:%S')
            url = item.link.text if item.link else "No URL"
            if 'SPY' in title.upper() or 'TSLA' in title.upper() or 'S&P 500' in title.upper():
                print(f"Adding Google News: {title} (Ticker: {'SPY' if 'SPY' in title.upper() or 'S&P 500' in title.upper() else 'TSLA'})")
                google_data.append({
                    'timestamp': timestamp, 'source': 'Google News',
                    'ticker': 'SPY' if 'SPY' in title.upper() or 'S&P 500' in title.upper() else 'TSLA',
                    'headline': title, 'url': url
                })
        print(f"Scraped {len(google_data)} Google News row(s)")
    except Exception as e:
        print(f"Google News scrape failed: {e}")
    return pd.DataFrame(google_data)

def scrape_yahoo_finance():
    print("\n=== Scraping Yahoo Finance ===")
    yahoo_data = []
    for ticker, url in [("SPY", "https://finance.yahoo.com/rss/headline?s=SPY"), ("TSLA", "https://finance.yahoo.com/rss/headline?s=TSLA")]:
        try:
            response = requests.get(url, timeout=30, headers={'User-Agent': 'Mozilla/5.0'})
            print(f"Yahoo {ticker} response status: {response.status_code}")
            soup = BeautifulSoup(response.text, 'xml')
            items = soup.find_all('item')[:5]
            print(f"Found {len(items)} Yahoo {ticker} articles")
            for item in items:
                title = item.title.text if item.title else "No title"
                pub_date = item.pubDate.text if item.pubDate else datetime.now(pytz.utc)
                if isinstance(pub_date, str):
                    try:
                        pub_date = datetime.strptime(pub_date, "%a, %d %b %Y %H:%M:%S %z")
                    except ValueError:
                        pub_date = datetime.strptime(pub_date, "%a, %d %b %Y %H:%M:%S GMT")
                        pub_date = pub_date.replace(tzinfo=pytz.UTC)
                pdt = pytz.timezone('America/Los_Angeles')
                timestamp = pub_date.astimezone(pdt).strftime('%Y-%m-%d %H:%M:%S')
                url = item.link.text if item.link else "No URL"
                if ticker in title.upper() or ('S&P 500' in title.upper() and ticker == 'SPY'):
                    print(f"Adding Yahoo {ticker}: {title}")
                    yahoo_data.append({
                        'timestamp': timestamp, 'source': 'Yahoo Finance',
                        'ticker': ticker, 'headline': title, 'url': url
                    })
            print(f"Scraped {len(yahoo_data)} Yahoo {ticker} row(s)")
        except Exception as e:
            print(f"Yahoo {ticker} scrape failed: {e}")
    return pd.DataFrame(yahoo_data)

def scrape_cnbc_news():
    print("\n=== Scraping CNBC ===")
    cnbc_data = []
    url = "https://www.cnbc.com/id/100003114/device/rss/rss.html"
    try:
        response = requests.get(url, timeout=30, headers={'User-Agent': 'Mozilla/5.0'})
        print(f"CNBC response status: {response.status_code}")
        soup = BeautifulSoup(response.text, 'xml')
        items = soup.find_all('item')[:10]
        print(f"Found {len(items)} CNBC RSS items")
        for item in items:
            title = item.title.text if item.title else "No title"
            pub_date = item.pubDate.text if item.pubDate else datetime.now(pytz.utc)
            if isinstance(pub_date, str):
                try:
                    pub_date = datetime.strptime(pub_date, "%a, %d %b %Y %H:%M:%S %z")
                except ValueError:
                    pub_date = datetime.strptime(pub_date, "%a, %d %b %Y %H:%M:%S GMT")
                    pub_date = pub_date.replace(tzinfo=pytz.UTC)
            pdt = pytz.timezone('America/Los_Angeles')
            timestamp = pub_date.astimezone(pdt).strftime('%Y-%m-%d %H:%M:%S')
            url = item.link.text if item.link else "No URL"
            description = item.description.text if item.description else ""
            if any(x in (title.upper() + description.upper()) for x in ['SPY', 'TSLA', 'S&P 500', 'TESLA']):
                ticker = 'SPY' if 'SPY' in (title.upper() + description.upper()) or 'S&P 500' in (title.upper() + description.upper()) else 'TSLA'
                print(f"Adding CNBC: {title} (Ticker: {ticker})")
                cnbc_data.append({
                    'timestamp': timestamp, 'source': 'CNBC',
                    'ticker': ticker, 'headline': title, 'url': url
                })
        print(f"Scraped {len(cnbc_data)} CNBC row(s)")
    except Exception as e:
        print(f"CNBC scrape failed: {e}")
    return pd.DataFrame(cnbc_data)

def scrape_seeking_alpha_news():
    print("\n=== Scraping Seeking Alpha ===")
    sa_data = []
    url = "https://seekingalpha.com/market_currents.xml"
    try:
        response = requests.get(url, timeout=30, headers={'User-Agent': 'Mozilla/5.0'})
        print(f"Seeking Alpha response status: {response.status_code}")
        soup = BeautifulSoup(response.text, 'xml')
        items = soup.find_all('item')[:10]
        print(f"Found {len(items)} Seeking Alpha RSS items")
        for item in items:
            title = item.title.text if item.title else "No title"
            pub_date = item.pubDate.text if item.pubDate else datetime.now(pytz.utc)
            if isinstance(pub_date, str):
                try:
                    pub_date = datetime.strptime(pub_date, "%a, %d %b %Y %H:%M:%S %z")
                except ValueError:
                    pub_date = datetime.strptime(pub_date, "%a, %d %b %Y %H:%M:%S GMT")
                    pub_date = pub_date.replace(tzinfo=pytz.UTC)
            pdt = pytz.timezone('America/Los_Angeles')
            timestamp = pub_date.astimezone(pdt).strftime('%Y-%m-%d %H:%M:%S')
            url = item.link.text if item.link else "No URL"
            description = item.description.text if item.description else ""
            if any(x in (title.upper() + description.upper()) for x in ['SPY', 'TSLA', 'S&P 500', 'TESLA']):
                ticker = 'SPY' if 'SPY' in (title.upper() + description.upper()) or 'S&P 500' in (title.upper() + description.upper()) else 'TSLA'
                print(f"Adding Seeking Alpha: {title} (Ticker: {ticker})")
                sa_data.append({
                    'timestamp': timestamp, 'source': 'Seeking Alpha',
                    'ticker': ticker, 'headline': title, 'url': url
                })
        print(f"Scraped {len(sa_data)} Seeking Alpha row(s)")
    except Exception as e:
        print(f"Seeking Alpha scrape failed: {e}")
    return pd.DataFrame(sa_data)

def scrape_nyt_business_news():
    print("\n=== Scraping NYT Business ===")
    nyt_data = []
    url = "https://rss.nytimes.com/services/xml/rss/nyt/Business.xml"
    try:
        response = requests.get(url, timeout=30, headers={'User-Agent': 'Mozilla/5.0'})
        print(f"NYT Business response status: {response.status_code}")
        soup = BeautifulSoup(response.text, 'xml')
        items = soup.find_all('item')[:10]
        print(f"Found {len(items)} NYT Business RSS items")
        for item in items:
            title = item.title.text if item.title else "No title"
            pub_date = item.pubDate.text if item.pubDate else datetime.now(pytz.utc)
            if isinstance(pub_date, str):
                try:
                    pub_date = datetime.strptime(pub_date, "%a, %d %b %Y %H:%M:%S %z")
                except ValueError:
                    pub_date = datetime.strptime(pub_date, "%Y-%m-%dT%H:%M:%S%z")
                    pub_date = pub_date.replace(tzinfo=pytz.UTC)
            pdt = pytz.timezone('America/Los_Angeles')
            timestamp = pub_date.astimezone(pdt).strftime('%Y-%m-%d %H:%M:%S')
            url = item.link.text if item.link else "No URL"
            description = item.description.text if item.description else ""
            if any(x in (title.upper() + description.upper()) for x in ['SPY', 'TSLA', 'S&P 500', 'TESLA']):
                ticker = 'SPY' if 'SPY' in (title.upper() + description.upper()) or 'S&P 500' in (title.upper() + description.upper()) else 'TSLA'
                print(f"Adding NYT Business: {title} (Ticker: {ticker})")
                nyt_data.append({
                    'timestamp': timestamp, 'source': 'NYT Business',
                    'ticker': ticker, 'headline': title, 'url': url
                })
        print(f"Scraped {len(nyt_data)} NYT Business row(s)")
    except Exception as e:
        print(f"NYT Business scrape failed: {e}")
    return pd.DataFrame(nyt_data)

def scrape_wapo_business_news():
    print("\n=== Scraping WaPo Business ===")
    wapo_data = []
    url = "https://feeds.washingtonpost.com/rss/business?itid=lk_inline_manual_32"
    try:
        response = requests.get(url, timeout=30, headers={'User-Agent': 'Mozilla/5.0'})
        print(f"WaPo Business response status: {response.status_code}")
        soup = BeautifulSoup(response.text, 'xml')
        items = soup.find_all('item')[:10]
        print(f"Found {len(items)} WaPo Business RSS items")
        for item in items:
            title = item.title.text if item.title else "No title"
            pub_date = item.pubDate.text if item.pubDate else datetime.now(pytz.utc)
            if isinstance(pub_date, str):
                try:
                    pub_date = datetime.strptime(pub_date, "%a, %d %b %Y %H:%M:%S %z")
                except ValueError:
                    pub_date = datetime.strptime(pub_date, "%Y-%m-%dT%H:%M:%SZ")
                    pub_date = pub_date.replace(tzinfo=pytz.UTC)
            pdt = pytz.timezone('America/Los_Angeles')
            timestamp = pub_date.astimezone(pdt).strftime('%Y-%m-%d %H:%M:%S')
            url = item.link.text if item.link else "No URL"
            description = item.description.text if item.description else ""
            if any(x in (title.upper() + description.upper()) for x in ['SPY', 'TSLA', 'S&P 500', 'TESLA']):
                ticker = 'SPY' if 'SPY' in (title.upper() + description.upper()) or 'S&P 500' in (title.upper() + description.upper()) else 'TSLA'
                print(f"Adding WaPo Business: {title} (Ticker: {ticker})")
                wapo_data.append({
                    'timestamp': timestamp, 'source': 'WaPo Business',
                    'ticker': ticker, 'headline': title, 'url': url
                })
        print(f"Scraped {len(wapo_data)} WaPo Business row(s)")
    except Exception as e:
        print(f"WaPo Business scrape failed: {e}")
    return pd.DataFrame(wapo_data)

def scrape_google_alerts_tsla():
    print("\n=== Scraping Google Alerts (TSLA) ===")
    tsla_data = []
    url = "https://www.google.com/alerts/feeds/08226079376422840598/3686544122646381017"
    try:
        response = requests.get(url, timeout=30, headers={'User-Agent': 'Mozilla/5.0'})
        print(f"Google Alerts TSLA response status: {response.status_code}")
        soup = BeautifulSoup(response.text, 'xml')
        items = soup.find_all('entry')[:10]
        print(f"Found {len(items)} Google Alerts TSLA RSS items")
        for item in items:
            title = item.title.text if item.title else "No title"
            pub_date = item.published.text if item.published else datetime.now(pytz.utc)
            if isinstance(pub_date, str):
                pub_date = datetime.strptime(pub_date, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=pytz.UTC)
            pdt = pytz.timezone('America/Los_Angeles')
            timestamp = pub_date.astimezone(pdt).strftime('%Y-%m-%d %H:%M:%S')
            link = item.link.get('href') if item.link and item.link.get('href') else "No URL"
            if link.startswith("https://www.google.com/url?"):
                parsed_url = urllib.parse.urlparse(link)
                query = urllib.parse.parse_qs(parsed_url.query)
                url = query.get('url', [link])[0]
            else:
                url = link
            if 'TSLA' in title.upper() or 'TESLA' in title.upper():
                print(f"Adding Google Alerts TSLA: {title}")
                tsla_data.append({
                    'timestamp': timestamp, 'source': 'Google Alerts',
                    'ticker': 'TSLA', 'headline': title, 'url': url
                })
        print(f"Scraped {len(tsla_data)} Google Alerts TSLA row(s)")
    except Exception as e:
        print(f"Google Alerts TSLA scrape failed: {e}")
    return pd.DataFrame(tsla_data)

def scrape_google_alerts_spy():
    print("\n=== Scraping Google Alerts (SPY) ===")
    spy_data = []
    url = "https://www.google.com/alerts/feeds/08226079376422840598/9371419328304257641"
    try:
        response = requests.get(url, timeout=30, headers={'User-Agent': 'Mozilla/5.0'})
        print(f"Google Alerts SPY response status: {response.status_code}")
        soup = BeautifulSoup(response.text, 'xml')
        items = soup.find_all('entry')[:10]
        print(f"Found {len(items)} Google Alerts SPY RSS items")
        for item in items:
            title = item.title.text if item.title else "No title"
            pub_date = item.published.text if item.published else datetime.now(pytz.utc)
            if isinstance(pub_date, str):
                pub_date = datetime.strptime(pub_date, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=pytz.UTC)
            pdt = pytz.timezone('America/Los_Angeles')
            timestamp = pub_date.astimezone(pdt).strftime('%Y-%m-%d %H:%M:%S')
            link = item.link.get('href') if item.link and item.link.get('href') else "No URL"
            if link.startswith("https://www.google.com/url?"):
                parsed_url = urllib.parse.urlparse(link)
                query = urllib.parse.parse_qs(parsed_url.query)
                url = query.get('url', [link])[0]
            else:
                url = link
            if 'SPY' in title.upper() or 'S&P 500' in title.upper() or 'SPDR S&P 500' in title.upper():
                print(f"Adding Google Alerts SPY: {title}")
                spy_data.append({
                    'timestamp': timestamp, 'source': 'Google Alerts',
                    'ticker': 'SPY', 'headline': title, 'url': url
                })
        print(f"Scraped {len(spy_data)} Google Alerts SPY row(s)")
    except Exception as e:
        print(f"Google Alerts SPY scrape failed: {e}")
    return pd.DataFrame(spy_data)



[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
import pandas as pd
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

LIVEWIRE_OUTPUT = "/content/drive/MyDrive/Livewire/all_x.csv"
LEXICON_PATH = "/content/drive/MyDrive/Livewire/stock_lexicon.csv"

def process_livewire(lexicon_df):
    print("\n=== Processing Livewire Data ===")
    scrape_functions = [
        scrape_google_news, scrape_yahoo_finance, scrape_cnbc_news,
        scrape_seeking_alpha_news, scrape_nyt_business_news, scrape_wapo_business_news,
        scrape_google_alerts_tsla, scrape_google_alerts_spy
    ]
    all_dfs = []
    for func in scrape_functions:
        try:
            df = func()
            print(f"{func.__name__} returned {len(df)} rows")
            if not df.empty:
                all_dfs.append(df)
        except Exception as e:
            print(f"{func.__name__} failed: {e}")

    all_df = pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame(
        columns=['timestamp', 'source', 'ticker', 'headline', 'url']
    )
    print(f"Combined rows before dedupe: {len(all_df)}")

    if all_df.empty:
        print("No data scraped—returning empty DataFrame")
        return all_df

    try:
        all_df['headline'] = all_df['headline'].str.replace(r'<[^>]+>|[^\w\s]', '', regex=True).str.strip()
        print("Duplicate check (pre-dedupe):")
        dups = all_df[all_df.duplicated(subset=['headline'], keep=False)]
        if not dups.empty:
            print(dups[['headline', 'url']].to_string(index=False))
        else:
            print("No duplicates found")
        deduped_df = all_df.drop_duplicates(subset=['headline'], keep='first').reset_index(drop=True)  # Initial dedupe
        print(f"Combined rows after dedupe: {len(deduped_df)}")

        def get_sentiment(headline, lexicon_df):
            blob = TextBlob(headline)
            vader_score = sid.polarity_scores(headline)['compound']
            base_score = (blob.sentiment.polarity + vader_score) / 2
            lexicon_score = 0
            matches = 0
            for _, row in lexicon_df.iterrows():
                if row['Item'] in headline.lower():
                    lexicon_score += row['Sentiment_Score']
                    matches += 1
            final_score = lexicon_score / matches if matches > 0 else base_score
            negative_keywords = ['fallout', 'red', 'drop', 'decline', 'crash', 'fall', 'plunge', 'challenges', 'downgrade', 'slide']
            positive_keywords = ['buy', 'surge', 'rally', 'up', 'bullish']
            pos_matches = any(kw in headline.lower() for kw in positive_keywords)
            neg_matches = any(kw in headline.lower() for kw in negative_keywords)
            if pos_matches:
                final_score = max(final_score, 0.3)
            elif neg_matches:
                final_score = min(final_score, -0.3)
            else:
                final_score = 0 if abs(final_score) < 0.1 else final_score
            sentiment = ('tense' if final_score < 0 and any(kw in headline.lower() for kw in ['crash', 'fall', 'plunge']) else
                        'shaky' if final_score < 0 else
                        'neutral' if final_score <= 0.5 else 'bullish')
            return sentiment, final_score

        deduped_df[['sentiment', 'sentiment_score']] = deduped_df['headline'].apply(lambda x: pd.Series(get_sentiment(x, lexicon_df)))
        deduped_df['timestamp'] = pd.to_datetime(deduped_df['timestamp'], errors='coerce')
        sorted_df = deduped_df.dropna(subset=['timestamp']).sort_values('timestamp', ascending=False)
        final_df = sorted_df.drop_duplicates(subset=['headline'], keep='first').head(15)  # Final dedupe after sort

        # Debug: Check for duplicates in final_df
        final_dups = final_df[final_df.duplicated(subset=['headline'], keep=False)]
        if not final_dups.empty:
            print("WARNING: Duplicates found in final output:")
            print(final_dups[['headline', 'url']].to_string(index=False))
        else:
            print("No duplicates in final output.")

        print("Combined sample (top 15):")
        print(final_df.to_string(index=False))
        final_df.to_csv(LIVEWIRE_OUTPUT, index=False)
        print(f"Saved {len(final_df)} rows to {LIVEWIRE_OUTPUT}")
        return final_df
    except Exception as e:
        print(f"Processing failed: {e}")
        return all_df

def make_decision(sentiment_score, capital=10000):
    dp = max(min(sentiment_score * 20, 10), -10)
    decision = "Buy" if dp > 3 else "Sell" if dp < -3 else "Hold"  # Adjusted thresholds
    net = 0.035 * capital if decision == "Buy" else -0.035 * capital if decision == "Sell" else 0
    return decision, dp, net



if __name__ == "__main__":
    print("=== Starting Full Livewire Pipeline ===")
    lexicon_df = pd.read_csv(LEXICON_PATH)
    print(f"Loaded lexicon with {len(lexicon_df)} terms")
    livewire_df = process_livewire(lexicon_df)
    print("\n=== Final Results with Algo Decisions ===")
    if livewire_df is None or livewire_df.empty:
        print("No data to process—skipping decisions")
    else:
        for _, row in livewire_df.iterrows():
            decision, dp, net = make_decision(row['sentiment_score'])
            print(f"Headline: {row['headline'][:50]}... | Sentiment: {row['sentiment']} | Score: {row['sentiment_score']:.2f} | dp: {dp} | Decision: {decision} | Net: {net}")

=== Starting Full Livewire Pipeline ===
Loaded lexicon with 612 terms

=== Processing Livewire Data ===

=== Scraping Google News ===
Google News response status: 200
Found 10 Google News RSS items
Adding Google News: Elon tells Tesla employees not to sell TSLA stocks as board and execs are dumping - Electrek.co (Ticker: TSLA)
Adding Google News: Netanyahu’s targeting of an Israeli spy chief and the attorney general - Al Jazeera English (Ticker: SPY)
Adding Google News: Elon Musk's Potential Margin Call - The Facts (NASDAQ:TSLA) - Seeking Alpha (Ticker: TSLA)
Adding Google News: Why Tesla Inc. (TSLA) Stock Crashed On Monday - Yahoo Finance (Ticker: TSLA)
Adding Google News: Google’s $32B Deal and a Silicon Valley Spy Scandal! - The Information (Ticker: SPY)
Adding Google News: Russian journalist target by Kremlin-backed spy ring speaks out - WBUR News (Ticker: SPY)
Adding Google News: Arsonist, Killer, Saboteur, Spy: While Trump Courts Him, Putin Is Escalating Russia’s Hybrid War Again

In [7]:
print(pd.read_csv("/content/drive/MyDrive/Livewire/stock_lexicon.csv").head())

        Item  POS  Sentiment_Score
0  's market  NaN        -0.054855
1   's trade  NaN        -0.099529
2   & market  NaN        -0.077722
3    & stock  NaN        -0.064574
4    + stock  NaN         0.545200


#IMPROVEMENTS BEGIN HERE

_